In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display

# configuration
video_dir = './input_videos'
tracking_data_dir = './tracking_data'
output_video_dir = './output_videos'

# Ensure all directories exist
dirs_to_check = [video_dir, tracking_data_dir, output_video_dir]
for dir in dirs_to_check:
    if not os.path.exists(dir):
        os.makedirs(dir)
        print(f"Directory created: {dir}")

# Function to select a video using an interactive widget
def interactive_select_video(video_dir):
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mov', '.MOV', '.mp4'))]
    if not video_files:
        print("No video files found in the directory.")
        return None
    video_selector = widgets.Dropdown(
        options=video_files,
        description='Select Video:',
        disabled=False,
    )
    display(video_selector)
    return video_selector

# Use the interactive selection component to select a video
video_selector = interactive_select_video(video_dir)


Dropdown(description='Select Video:', options=('20220520_150630.mp4', 'test_squat.mp4', 'IMG_9291.MOV'), value…

In [6]:

from tracking.video_processing_pipeline import process_video, body_parts

# Track the selected video
try:
    selected_video = video_selector.value
except AttributeError:
    print("No video was selected.")
    selected_video = None

if selected_video:
    # Process the selected video
    process_video(selected_video, video_dir, tracking_data_dir)

    # set the path to the tracking data file
    tracking_data_file = f'Tracking_{selected_video[:-4]}.csv'

INFO:root:Processing video: IMG_9291.MOV
INFO:root:Tracking data for IMG_9291.MOV stored in ./tracking_data/Tracking_IMG_9291.csv


In [7]:
# Generate Video Tracking Preview
import pandas as pd
import os
import cv2
import mediapipe as mp
from IPython.display import Video, HTML

# Function to save video with landmarks from tracking data
def save_landmarks_on_video(video_path, output_video_path, tracking_data_path):
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get the width, height, and frame rate of the video frame
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Read the tracking data from CSV
    tracking_data = pd.read_csv(tracking_data_path)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'avc1')  # or 'XVID'
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Process video and draw landmarks from tracking data
    for index, row in tracking_data.iterrows():
        # Read frame from video
        ret, frame = cap.read()
        if not ret:
            break

        # Draw landmarks on the frame
        for landmark in mp.solutions.pose.PoseLandmark:
            landmark_name = landmark.name.lower().replace('_', ' ')  # Convert to lowercase and replace underscores with spaces
            landmark_name = landmark_name.capitalize()  # Capitalize only the first letter of the first word
            landmark_x = f'{landmark_name}_x'
            landmark_y = f'{landmark_name}_y'
            if landmark_x in tracking_data.columns and landmark_y in tracking_data.columns and not pd.isna(row[landmark_x]) and not pd.isna(row[landmark_y]):
                x = int(row[landmark_x] * width)
                y = int(row[landmark_y] * height)
                # Check if the current landmark is one of the hips
                if landmark in [mp.solutions.pose.PoseLandmark.LEFT_HIP, mp.solutions.pose.PoseLandmark.RIGHT_HIP]:
                    # Highlight hips in red
                    cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
                else:
                    # Draw other landmarks in green
                    cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)


        # Write the frame with landmarks to the output video file
        out.write(frame)

    # Release everything when job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Video saved to {output_video_path}")

# Call the function to save the video with landmarks
video_path = os.path.join(video_dir, selected_video)

# Prepare the output video path
video_name = os.path.basename(video_path)
output_video_name = os.path.splitext(video_name)[0] + "_landmarks.mp4"
output_video_path = os.path.join(output_video_dir, output_video_name)
tracking_data_path = os.path.join(tracking_data_dir, tracking_data_file)
save_landmarks_on_video(video_path, output_video_path, tracking_data_path)

# Output
# Create an HTML string with the video embedded
video_html = f'''
Video saved to 
<a href='{output_video_path}' target='_blank'>{output_video_path}</a> <br><br>
<video height="500" controls allowfullscreen>
  <source src="{output_video_path}" type="video/mp4">
Your browser does not support the video tag.
</video>
'''
display(HTML(video_html))

Video saved to ./output_videos/IMG_9291_landmarks.mp4
